In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
from time import sleep

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://hh.ru/search/vacancy?area=68&search_field=name&search_field=company_name&search_field=description&text=аналитик+баз+данных&no_magic=true&L_save_area=true&items_on_page=20'


In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

In [4]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
        print('Парсинг завершен')
        return result
    else:
        print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
        result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
        link_next_page =  'https://hh.ru' + link_next_page['href']
    else:
        print('Парсинг завершен')
        return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result

def parse_vacancy_hh(dom_vacancy):
    sleep(0.1)

    vacancy_name = dom_vacancy.find('a', {'class': 'serp-item__title'}).getText()
    
    vacancy_salary = dom_vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if vacancy_salary:
        vacancy_salary = vacancy_salary.text.replace('\u202f', '')
        min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
    else:
        min_salary, max_salary, currency_salary = None, None, None 
        
    vacancy_link = dom_vacancy.find('a', {'class': 'serp-item__title'})['href']
                
    return {
        'vacancy_name': vacancy_name,
        'vacancy_salary': vacancy_salary,
        'min_salary': min_salary,
        'max_salary': max_salary,
        'currency_salary': currency_salary,
        'vacancy_link': vacancy_link,
        'vacancy_source': 'hh.ru',
    }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
    list_salary = vacancy_salary_text.replace('\u202f', '').split()
    for i in range(len(list_salary) - 1):
        if list_salary[i] == 'от':
            min_salary = int(list_salary[i + 1])
        elif list_salary[i] == 'до':
            max_salary = int(list_salary[i + 1])
        elif list_salary[i] == '–':
            min_salary = int(list_salary[i - 1])
            max_salary = int(list_salary[i + 1])
    currency_salary = list_salary[-1]

    return min_salary, max_salary, currency_salary
  

In [5]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://hh.ru/search/vacancy?area=68&search_field=name&search_field=company_name&search_field=description&text=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA+%D0%B1%D0%B0%D0%B7+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85&no_magic=true&L_save_area=true&items_on_page=20
Парсинг завершен


In [6]:
len(result)

11

In [7]:
result

[{'vacancy_name': 'ГИС программист (разработка/аналитика/системное администрирование)',
  'vacancy_salary': '90000 – 180000 руб.',
  'min_salary': 90000,
  'max_salary': 180000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://hh.ru/vacancy/77446643?from=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA+%D0%B1%D0%B0%D0%B7+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Маркетолог-аналитик',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://hh.ru/vacancy/72668796?from=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA+%D0%B1%D0%B0%D0%B7+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Аналитик 1С / Консультант 1С',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://hh.ru/vacancy/77019393?from=vacancy_search_li

1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [8]:
!pip install pymongo

In [9]:
import pymongo
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from pprint import pprint

In [10]:
client = MongoClient('localhost', 27017)

In [11]:
print(client.list_database_names()) 

['admin', 'config', 'local', 'vacancies_dict']


In [12]:
db = client.vacancies_dict

In [13]:
collection = db.vacancies

In [14]:
collection.delete_many({})

In [15]:
try:
    collection.insert_many(result)
except DuplicateKeyError:
    print('Объект с _id:\t%s уже записан в колекцию'%doc['_id'])

In [16]:
print(client.list_database_names())

['admin', 'config', 'local', 'vacancies_dict']


In [17]:
for doc in collection.find():
    pprint(doc)

{'_id': ObjectId('63fdd58f0ac570b367b13fbf'),
 'currency_salary': 'руб.',
 'max_salary': 180000,
 'min_salary': 90000,
 'vacancy_link': 'https://hh.ru/vacancy/77446643?from=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA+%D0%B1%D0%B0%D0%B7+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85',
 'vacancy_name': 'ГИС программист (разработка/аналитика/системное '
                 'администрирование)',
 'vacancy_salary': '90000 – 180000 руб.',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63fdd58f0ac570b367b13fc0'),
 'currency_salary': None,
 'max_salary': None,
 'min_salary': None,
 'vacancy_link': 'https://hh.ru/vacancy/72668796?from=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA+%D0%B1%D0%B0%D0%B7+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85',
 'vacancy_name': 'Маркетолог-аналитик',
 'vacancy_salary': None,
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63fdd58f0ac570b367b13fc1'),
 'currency_salary': None,
 'max_salary': None,
 'min_salary': None,
 'vacancy

In [18]:
def cheak_and_save_vacancies_in_db(vacancies):
    for vacancy in vacancies:
        if not len(list(collection.find({'vacancy_link': vacancy['vacancy_link']}))):
            collection.insert_one(vacancy)

In [19]:
cheak_and_save_vacancies_in_db(result)

In [20]:
result_find = list(collection.find())

In [21]:
len(result_find)

11

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).

In [22]:
def cheak_vacancy_salary_in_db(database, salary_level):
    vacancies = database.find()
    for vacancy in vacancies:
        if type(vacancy['max_salary']) is int and type(vacancy['min_salary']) is int:
            if int(salary_level) < vacancy['max_salary'] or int(salary_level) < vacancy['min_salary']:
                pprint('Вакансия %s с зарплатой от %.2f до %.2f' %(vacancy['vacancy_name'],vacancy['min_salary'], vacancy['max_salary']))

In [23]:
cheak_vacancy_salary_in_db(collection, 80000)

('Вакансия ГИС программист (разработка/аналитика/системное администрирование) '
 'с зарплатой от 90000.00 до 180000.00')
('Вакансия Сеньор системный аналитик/архитектор ПО с зарплатой от 150000.00 до '
 '300000.00')
